In [2]:

import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

def euler_from_rvec(rvec):
    R, _ = cv2.Rodrigues(rvec)
    
    # Calculate Euler angles (roll, pitch, yaw)
    pitch = np.arctan2(R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2))
    roll = np.arctan2(R[2, 1], R[2, 2])
    yaw = np.degrees(np.arctan2(R[1, 0], R[0, 0]))
    if yaw >-180 and yaw < 0:
        yaw=abs(yaw)
    else:
        yaw=360-yaw
        
    return roll, pitch, yaw
 
def draw_trajectory(img, odom_position, color=(0, 255, 255), thickness=10):
    for k in range(1, len(odom_position)):
        start_point = tuple(odom_position[k-1][:2])  # First two elements of the previous position
        end_point = tuple(odom_position[k][:2])     # First two elements of the current position
        cv2.line(img, start_point, end_point, color, thickness)

def update_odom_position(odom_tvec, marker_corners_3d, newcameramtx, dist, period,centers,yaw,initial,flag):
    
    odom_rvec=  np.array([[0.], [0.], [0.]])
    #odom_tvec,yaw = update_position(odom_tvec, odom_rvec,period,yaw,initial) 
    #next_time, x_updated, P_updated, x_predicted = kf.kalman_filter(flag, u, z)
    
    # Convert odometry vectors to the appropriate format
    odom_tvec = np.array(odom_tvec, dtype=np.float64).reshape(3, 1)
    odom_rvec = np.array(odom_rvec, dtype=np.float64).reshape(3, 1)


    
    projected_points_odom, _ = cv2.projectPoints(marker_corners_3d,odom_rvec,odom_tvec,newcameramtx, dist)
    projected_points_odom = projected_points_odom.reshape(-1, 2).astype(int)

    # Compute the center of the projected points
    center_odom = np.mean(projected_points_odom, axis=0).astype(int)
    centers.append(center_odom)
    return odom_tvec, odom_rvec, center_odom, period,centers,yaw
import cv2

def draw_marker_paths(img, marker_centers, marker_id, center, circle_color=(0, 0, 255), line_color=(255, 0, 0), circle_radius=5, line_thickness=2, window_name="Marker Path Tracking"):

    cv2.circle(img, tuple(center), circle_radius, circle_color, -1)
    if len(marker_centers[marker_id]) > 1:
        for j in range(1, len(marker_centers[marker_id])):
            cv2.line(img, tuple(marker_centers[marker_id][j-1]), tuple(marker_centers[marker_id][j]), line_color, line_thickness)

    # Display the output
    cv2.imshow(window_name, img)
import cv2
import numpy as np
import yaml
from cv2 import aruco

def initialize_camera_and_calibration(calibration_file = 'calibration.yaml', camera_index=1, aruco_dict_type=aruco.DICT_6X6_1000):
    with open(calibration_file, 'r') as f:
        calibration_data = yaml.safe_load(f)

    mtx = np.array(calibration_data['camera_matrix'])
    dist = np.array(calibration_data['dist_coeff'])

    # Load the ArUco dictionary and parameters
    aruco_dict = aruco.getPredefinedDictionary(aruco_dict_type)
    aruco_params = aruco.DetectorParameters()

    # Initialize the camera
    camera = cv2.VideoCapture(camera_index)
    if not camera.isOpened():
        raise RuntimeError("Error: Could not open the camera.")

    # Read a frame to determine the image size
    ret, img = camera.read()
    if not ret:
        raise RuntimeError("Error: Could not read from the camera.")

    h, w = img.shape[:2]

    # Get the optimal camera matrix
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

    return camera, aruco_dict, aruco_params, newcameramtx, dist, mtx
def detect_marker_with_id(frame, aruco_dict, aruco_params, target_id=10):
    robot_detected=False
    # Detect markers in the frame
    corners, ids, rejected_img_points = aruco.detectMarkers(frame, aruco_dict, parameters=aruco_params)

    if ids is not None:
        for i, marker_id in enumerate(ids.flatten()):
            if marker_id == target_id:
                
                return corners[i], marker_id, True

    # Return None if the target ID is not found
    return None,None,False
def process_marker(corners, marker_id, target_id, camera_matrix, dist_coeffs,marker_centers):
    
    if marker_id != target_id:
        return None, None, None, None

    # Get the 2D positions of the marker corners
    marker_2d = corners[0]

    # Define the 3D positions of the marker corners in the world frame
    marker_corners_3d = np.array([
        [-0.45, -0.45, 0],
        [0.45, -0.45, 0],
        [0.45, 0.45, 0],
        [-0.45, 0.45, 0]
    ], dtype=np.float32)

    # Solve PnP to find rotation and translation vectors
    _, rvec_marker, tvec_marker = cv2.solvePnP(marker_corners_3d, marker_2d, camera_matrix, dist_coeffs)

    # Project the 3D points to 2D
    projected_points, _ = cv2.projectPoints(marker_corners_3d, rvec_marker, tvec_marker, camera_matrix, dist_coeffs)
    projected_points = projected_points.reshape(-1, 2).astype(int)

    # Calculate the center of the marker
    center = tuple(np.mean(projected_points, axis=0).astype(int))
    marker_centers.append(center)
    
    return center, projected_points, tvec_marker, rvec_marker, marker_centers,marker_corners_3d
def draw_marker_path(image, marker_centers, color=(255, 0, 0), thickness=2):
    cv2.circle(image, tuple(center), 5, (0, 0, 255), -1)

    if len(marker_centers) > 1:
        for j in range(1, len(marker_centers)):
            cv2.line(image, tuple(marker_centers[j-1]), tuple(marker_centers[j]), color, thickness)
    return image
def get_z(rvec,center):
    roll,pitch,yaw = euler_from_rvec(rvec)
    
    return [center[0],center[1],np.round((yaw))]
    
def update_position(tvec,period,theta,initial_tz):
    period=np.round(period,8)
    #print(period)
    '''
    if period < 0.000001:
        time.sleep(0.000001-period)
    '''
    if (period < 0.02):  
        dt = 0.02
    dt=period*(conversion_factor/2)
    
    #rvec = np.array(rvec, dtype=np.float64)
    # Unpack current position (x, y, theta)
    #dt=0.0001
    #v = (motor_left_speed + motor_right_speed) / 2
    #v =v/0.3870967741935484
    #ls=motor_left_speed
    #rs=motor_right_speed
    ls,rs=speeds()
    #print(ls,rs)
    #print(tvec)
    x, y,_= tvec
    robot_length=1.1
    #theta=get_theta_from_rvec_and_tvec(tvec,rvec)
    #theta=0
    # Compute the change in position using the odometry equations
    delta_x = (0.5 * dt * np.cos(theta) * ls + 0.5 * dt * np.cos(theta) * rs )
    delta_y =(-0.5 * dt * np.sin(theta) * ls + -0.5 * dt * np.sin(theta) * rs )
    delta_theta = (-dt / robot_length) * ls + (dt / robot_length) * rs
    #print(delta_theta)
    new_theta=delta_theta+theta
    print(np.round(np.degrees(new_theta),3))
    updated_tvec = [x +delta_x, y + delta_y,np.array([ initial_tz])]
    return updated_tvec,new_theta

def warp_and_threshold(frame, rect_corners, width=600, height=800):

    # Define destination points for the flattened rectangle
    dst_corners = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype=np.float32)

    # Compute the perspective transform matrix
    matrix = cv2.getPerspectiveTransform(rect_corners, dst_corners)

    # Apply perspective warp to get the top-down view
    warped_image = cv2.warpPerspective(frame, matrix, (width, height))

    return warped_image
import cv2

def resize_image(image, new_width=60):

    # Get the original dimensions of the image
    height, width = image.shape[:2]
    
    # Calculate the new height to maintain the aspect ratio
    new_height = int((new_width / width) * height)
    
    # Resize the image using nearest neighbor interpolation
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
    
    return resized_image
import numpy as np
import cv2

def map_and_draw_path(image,matplotlib_points, matplotlib_size, opencv_size, image_color=(0, 0, 0), path_color=(255, 0, 0), thickness=2):
    
    matplotlib_width, matplotlib_height = matplotlib_size
    opencv_width, opencv_height = opencv_size
    

    # Scale points to OpenCV space (invert y-axis)
    points_cv = [
        (int(x / matplotlib_width * opencv_width), int(opencv_height - (y / matplotlib_height * opencv_height)))
        for x, y in matplotlib_points
    ]

    # Draw the path
    for i in range(1, len(points_cv)):
        cv2.line(image, points_cv[i - 1], points_cv[i], path_color, thickness)

    return image



In [3]:
camera, aruco_dict, aruco_params, newcameramtx, dist, mtx =initialize_camera_and_calibration()
odom_position=[]
first=False
odom_tvec=[0.,0.,0.]
# Processing loop
current_time=time.time()

period=0
yaw_odom=0
yaw_aruco=0
marker_centers=[]
odom_centers=[]
z=[0,0,0]
initial=False
yaw=0
kidnapped=False
conversion_factor = 0.3870967741935484 
odom_tvec=0
odom_rvec=np.array([[0.],[0.],[0.]])
tvec_marker=np.array([[0.],[0.],[0.]])
initial_tz=0
rect_corners = np.array([[125, 123], [544, 128], [582,340], [89, 343]], dtype=np.float32)

import time
marker_corners_3d=0
print("Press 'q' to exit.")
while True:
    ret, img = camera.read()
    if not ret:
        print("Error: Could not read from the camera.")
        break

    # Convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Undistort the image
    dst = cv2.undistort(img_gray, mtx, dist, None, newcameramtx)
    corners,marker_id, robot_detected = detect_marker_with_id(dst, aruco_dict, aruco_params, target_id=5)

    if robot_detected:
        # Draw the detected marker
        aruco.drawDetectedMarkers(img, [corners],np.array([marker_id]), (0, 255, 0))
        center, projected_points, tvec_marker, rvec_marker ,marker_centers,marker_corners_3d= process_marker(corners, marker_id, 5, newcameramtx, dist,marker_centers)
        z = get_z( rvec_marker,center )
        
        
        if not initial or kidnapped :
            
            odom_tvec = tvec_marker
            odom_centers.append(center)
            initial_tz=odom_tvec[2][0]
            odom_tvec, odom_rvec, center_odom, period,odom_centers,yaw_odom = update_odom_position(odom_tvec, marker_corners_3d, newcameramtx, dist, period,odom_centers,0,initial_tz)
            initial=True
            
            end_time = time.time()
            period = end_time - current_time
    else:
        print("robot not found.")        
    if initial:
        
        current_time=time.time()
        odom_tvec, odom_rvec, center_odom, period,odom_centers ,yaw_odom= update_odom_position(odom_tvec, marker_corners_3d, newcameramtx, dist, period,odom_centers,yaw_odom,initial_tz)
        
        end_time = time.time()
        period = end_time - current_time
            


    # Show the frame
    draw_trajectory(img,odom_centers)
    cv2.putText(img, "yaw aruco"+str(z[2]), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(img, "yaw"+str(np.degrees(yaw_odom)), (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    draw_marker_path(img, marker_centers)
    
    cv2.imshow("ArUco Detection", img)



    '''
    warped=warp_and_threshold(img,rect_corners)
   
    for i in range(1, len(path)):
        cv2.line(warped, tuple(path[i - 1]), tuple(path[i]), (255, 0, 0), 1)

    
    matplotlib_size = (800, 600)
    opencv_size = (800, 600)
    
    # Generate the image
    path_image = map_and_draw_path(warped,path, matplotlib_size, opencv_size)
    cv2.imshow("warped",warped )
    '''
    if cv2.waitKey(1) & 0xFF == ord('q'):
        motors(0, 0, verbose=True)
        break
    time.sleep(0.0003)
#camera.release()
cv2.destroyAllWindows()


NameError: name 'points' is not defined